In [1]:
%matplotlib widget
import os
import sys
import numpy as np
import matplotlib
import serial
import pyvisa
import csv
import struct as struct
from time import sleep, time
from pylab import *
from datetime import datetime

from scipy import integrate
from scipy.special import jv

import pylab as pyplt
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.interpolate import interp2d

import codecs
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm

In [2]:
def writeAUTData2File_3D(filepath, filenamenotype, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if(pol == 0): #p-polarized
                    azimuthDeg_p_pol.append(float(line[0:9]))
                    elevationDeg_p_pol.append(float(line[10:19]))
                    ampdb_p_pol.append(float(line[20:29]))
                    phaseDeg_p_pol.append(float(line[30:]))
                elif(pol > 0): #x-polarized
                    azimuthDeg_x_pol.append(float(line[0:9]))
                    elevationDeg_x_pol.append(float(line[10:19]))
                    ampdb_x_pol.append(float(line[20:29]))
                    phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                pol += 1
                next(f)
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_p_pol.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_p_pol.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_p_pol.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_p_pol.csv", phaseDeg_p_pol, delimiter=",")
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_x_pol.csv", azimuthDeg_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_x_pol.csv", elevationDeg_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_x_pol.csv", ampdb_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_x_pol.csv", phaseDeg_x_pol, delimiter=",")
            
def writeAUTData2File_2D(filepath, filenamenotype, elevation, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (elevation == float(line[10:19])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    np.savetxt(filepath + filenamenotype + "/azimuthDeg_p_pol_" + str(elevation) + "_elevation.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_p_pol_" + str(elevation) + "_elevation.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_p_pol_" + str(elevation) + "_elevation.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_p_pol_" + str(elevation) + "_elevation.csv", phaseDeg_p_pol, delimiter=",")
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_x_pol_" + str(elevation) + "_elevation.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_x_pol_" + str(elevation) + "_elevation.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_x_pol_" + str(elevation) + "_elevation.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_x_pol_" + str(elevation) + "_elevation.csv", phaseDeg_p_pol, delimiter=",")
            
        
def returnAUTData_3D(filepath, filenamenotype, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if(pol == 0): #p-polarized
                    azimuthDeg_p_pol.append(float(line[0:9]))
                    elevationDeg_p_pol.append(float(line[10:19]))
                    ampdb_p_pol.append(float(line[20:29]))
                    phaseDeg_p_pol.append(float(line[30:]))
                elif(pol > 0): #x-polarized
                    azimuthDeg_x_pol.append(float(line[0:9]))
                    elevationDeg_x_pol.append(float(line[10:19]))
                    ampdb_x_pol.append(float(line[20:29]))
                    phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                pol += 1
                next(f)
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)
    
    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol
            
def returnAUTData_2D(filepath, filenamenotype, elevation, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (elevation == float(line[10:19])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol
            
def returnAUTData_2D_septum_reverse(filepath, filenamenotype, azimuth, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (azimuth == float(line[0:9])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol

In [286]:
#writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15")
#writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15")
writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 79)
writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 79)

In [282]:
#writeAUTData2File_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0)
#writeAUTData2File_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0)
writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 81)

In [6]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 2U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 10, 79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 2U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", -5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", -10, 79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-40,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_ylim(-40,0)
axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles
#plt.rcParams['axes.titlepad'] = 10

fig.suptitle(r'Hermes 2U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 2U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 10, 79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 2U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", -5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", -10, 79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-40,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles
#plt.rcParams['axes.titlepad'] = 10

fig.suptitle(r'Hermes 2U - Vertical Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 10,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", -5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", -10,79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-10,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles

fig.suptitle(r'Hermes 3U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 10,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", -5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", -10,79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-10,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles

fig.suptitle(r'Hermes 3U - Vertical Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8.5
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

#plots
cmap = cm.get_cmap('viridis', 4)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_15 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3))

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_15 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3))

#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

p_pol_0 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')
p_pol_15 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(2),label=r'15$^\circ$')


x_pol_0 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')
x_pol_15 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(2),label=r'15$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -15,81)

#plots
p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_15 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3),label=r'-10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_15 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3),label=r'-10$^\circ$')

#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

#plots
p_pol_0 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'-0$^\circ$')
p_pol_5 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_15 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3),label=r'-15$^\circ$')

x_pol_0 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'-0$^\circ$')
x_pol_5 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_15 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3),label=r'-15$^\circ$')

#limits
axes[0,0].set_ylim(-5,0)
axes[0,0].set_yticks(np.arange(-5,0,1))
axes[0,1].set_ylim(-5,0)
axes[0,1].set_yticks(np.arange(-5,0,1))

axes[1,1].set_ylim(-5,0)
axes[1,0].set_yticks(np.arange(-5,0,1))
axes[1,1].set_ylim(-5,0)
axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
axes[0,1].legend()

#titles

fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_xlabel(r'Azimuth', fontsize = labelfont,labelpad=10)
axes[1,1].set_xlabel(r'Elevation', fontsize = labelfont,labelpad=10)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
##############################Initialise#####################################################
nrow = 1
ncol = 2
xsize = 8
ysize = 4.3
plt.close('all')
pi = 3.14159

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

fig.subplots_adjust(hspace=0.025, wspace=0.005)

fig.set_facecolor('w')

#x_locator = FixedLocator([lambda_s_center-lambda_s_width/4, lambda_s_center, lambda_s_center+lambda_s_width/4])
#x_formatter = FixedFormatter([0,r'$\pi/4$',r'$\pi/2$'])
#y_locator = FixedLocator([lambda_i_center-lambda_i_width/4, lambda_i_center, lambda_i_center+lambda_i_width/4])
#y_formatter = FixedFormatter([791.6,792.1,792.6])
x_locator = FixedLocator([-15, -10, -5, 0, 5, 10, 15])
#y_locator = FixedLocator([791.5, 792, 792.5])

for i in range(0,len(axes)):
    axes[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    axes[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
#    axes[i].xaxis.set_major_formatter(x_formatter)
    axes[i].xaxis.set_major_locator(x_locator)
    #axes[i].yaxis.set_major_formatter(y_formatter)
    axes[i].yaxis.set_major_locator(x_locator)
#    axes[i].set_xlim(-15,105)
#    axes[i].set_ylim(-0.1,2.6)
    
axes[1].set_yticklabels([])

#labels

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15",81)
ampdb = zeros((40,40))
ampdb_x = zeros((40,40))
azimuth = zeros(40)
for i in range(40):
    azimuth[i] = (-15+15/20*i)
    for j in range(40):
        theta = (-15+15/20*j)*pi/180
        phi = (-15 + 15/20*i)*pi/180
        #scale = 10*log10((np.cos(theta)**2)*(np.cos(phi)**2))
        #print(scale)
        #ampdb[39-i,j] = 10*log10(scale)# + ampdb_p_pol_0[j+40*i]
        ampdb[39-i,j] = ampdb_p_pol_0[j+40*i]# + scale
        ampdb_x[39-i,j] = ampdb_x_pol_0[j+40*i]# + scale
        
        
#plots
#axes[0].pcolormesh(azimuthDeg_p_pol_0[0:40], azimuthDeg_p_pol_0[0:40], ampdb, cmap='viridis')
axes[0].pcolormesh(azimuth, azimuth, ampdb, cmap='viridis')
axes[1].pcolormesh(azimuth, azimuth, ampdb_x, cmap='viridis')

#limits
#axes[0,0].set_ylim(-5,0)
#axes[0,0].set_yticks(np.arange(-5,0,1))
#axes[0,1].set_ylim(-5,0)
#axes[0,1].set_yticks(np.arange(-5,0,1))

#axes[1,1].set_ylim(-5,0)
#axes[1,0].set_yticks(np.arange(-5,0,1))
#axes[1,1].set_ylim(-5,0)
#axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
#axes[0,1].legend()

#titles

#fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0].set_ylabel(r'Elevation ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[0].set_xlabel(r'Azimuth - P Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[1].set_xlabel(r'Azimuth - X Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
##############################Initialise#####################################################
nrow = 1
ncol = 2
xsize = 8
ysize = 4.3
plt.close('all')
pi = 3.14159

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

fig.subplots_adjust(hspace=0.025, wspace=0.005)

fig.set_facecolor('w')

#x_locator = FixedLocator([lambda_s_center-lambda_s_width/4, lambda_s_center, lambda_s_center+lambda_s_width/4])
#x_formatter = FixedFormatter([0,r'$\pi/4$',r'$\pi/2$'])
#y_locator = FixedLocator([lambda_i_center-lambda_i_width/4, lambda_i_center, lambda_i_center+lambda_i_width/4])
#y_formatter = FixedFormatter([791.6,792.1,792.6])
#x_locator = FixedLocator([-15, -10, -5, 0, 5, 10, 15])
#y_locator = FixedLocator([791.5, 792, 792.5])

for i in range(0,len(axes)):
    axes[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    axes[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
#    axes[i].xaxis.set_major_formatter(x_formatter)
#    axes[i].xaxis.set_major_locator(x_locator)
    #axes[i].yaxis.set_major_formatter(y_formatter)
#    axes[i].yaxis.set_major_locator(x_locator)
#    axes[i].set_xlim(-15,105)
#    axes[i].set_ylim(-0.1,2.6)
    


#labels

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15",81)
ampdb = zeros((40,40))
ampdb_x = zeros((40,40))
azimuth = zeros(40)
for i in range(40):
    azimuth[i] = (-15+15/20*i)
    for j in range(40):
        theta = (-15+15/20*j)*pi/180
        phi = (-15 + 15/20*i)*pi/180
        #scale = 10*log10((np.cos(theta)**2)*(np.cos(phi)**2))
        #print(scale)
        #ampdb[39-i,j] = 10*log10(scale)# + ampdb_p_pol_0[j+40*i]
        ampdb[39-i,j] = ampdb_p_pol_0[j+40*i]# + scale
        ampdb_x[39-i,j] = ampdb_x_pol_0[j+40*i]# + scale
        
        
#plots
axes[0].plot([-15,15],[-3,-3],'--k')
axes[1].plot([-15,15],[-3,-3],'--k')
p_pol_0 = axes[0].plot(azimuth, ampdb[19], color = 'red')
x_pol_0 = axes[1].plot(azimuth, ampdb_x[19], color = 'red')
axes[0].plot([-15.1,-15.1],[-5,0],'--k')
axes[1].plot([-14.94,-14.94],[-5,0],'--k')
axes[0].plot([12.54,12.54],[-5,0],'--k')
axes[1].plot([13.9,13.9],[-5,0],'--k')
#limits
print(str(12.54+15.1))
print(str(14.94+13.9))
#axes[0,0].set_ylim(-5,0)
#axes[0,0].set_yticks(np.arange(-5,0,1))
#axes[0,1].set_ylim(-5,0)
#axes[0,1].set_yticks(np.arange(-5,0,1))

axes[1].set_ylim(-4,0)
axes[0].set_ylim(-4,0)
axes[1].set_xlim(-17,17)
axes[0].set_xlim(-17,17)
#axes[1,0].set_yticks(np.arange(-5,0,1))
#axes[1,1].set_ylim(-5,0)
#axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
#axes[0,1].legend()

#titles

#fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0].set_ylabel(r'Elevation ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[1].set_ylabel(r'Elevation ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[0].set_xlabel(r'Azimuth - P Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[1].set_xlabel(r'Azimuth - X Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

27.64
28.84


In [75]:
#integrating
result = zeros(len(arange(-10,10,0.1)))
elevation = arange(-100,100,1)
for scannedLine in range(len(arange(-10,10,0.1))):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", elevation[scannedLine]/10,79)
        amp = zeros(len(ampdb_p_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp[i] = 10**(ampdb_p_pol_0[i]/10)
        result[scannedLine] =max(ampdb_p_pol_0)-10*log10(np.trapz(amp, azimuthDeg_p_pol_0)/360*2)
    except:
        print(scannedLine)
        print(elevation[scannedLine])
print(np.mean(result))

for scannedLine_x in range(len(arange(-10,10,0.1))):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", elevation[scannedLine_x]/10,79)
        amp = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_x_pol_0)):
            amp[i] = 10**(ampdb_x_pol_0[i]/10)
        result_x[scannedLine_x] =max(ampdb_x_pol_0)-10*log10(np.trapz(amp, azimuthDeg_x_pol_0)/360*2)
    except:
        print(scannedLine_x)
        print(elevation[scannedLine_x])
print(np.mean(result_x))
print(10*log10(10**(np.mean(result_x)/10) + 10**(np.mean(result)/10)))

3.306044780009472
0
-100
1
-99
2
-98
3
-97
4
-96
5
-95
6
-94
7
-93
8
-92
9
-91
10
-90
11
-89
12
-88
13
-87
14
-86
15
-85
16
-84
17
-83
18
-82
19
-81
20
-80
21
-79
22
-78
23
-77
24
-76
25
-75
26
-74
27
-73
28
-72
29
-71
30
-70
31
-69
32
-68
33
-67
34
-66
35
-65
36
-64
37
-63
38
-62
39
-61
40
-60
41
-59
42
-58
43
-57
44
-56
45
-55
46
-54
47
-53
48
-52
49
-51
50
-50
51
-49
52
-48
53
-47
54
-46
55
-45
56
-44
57
-43
58
-42
59
-41
60
-40
61
-39
62
-38
63
-37
64
-36
65
-35
66
-34
67
-33
68
-32
69
-31
70
-30
71
-29
72
-28
73
-27
74
-26
75
-25
76
-24
77
-23
78
-22
79
-21
80
-20
81
-19
82
-18
83
-17
84
-16
85
-15
86
-14
87
-13
88
-12
89
-11
90
-10
91
-9
92
-8
93
-7
94
-6
95
-5
96
-4
97
-3
98
-2
99
-1
100
0
101
1
102
2
103
3
104
4
105
5
106
6
107
7
108
8
109
9
110
10
111
11
112
12
113
13
114
14
115
15
116
16
117
17
118
18
119
19
120
20
121
21
122
22
123
23
124
24
125
25
126
26
127
27
128
28
129
29
130
30
131
31
132
32
133
33
134
34
135
35
136
36
137
37
138
38
139
39
140
40
141
41
142
42
143
43
14

NameError: name 'result_x' is not defined

In [48]:
result

array([-20.42504379, -20.41792908, -20.41173218, -20.40465195,
       -20.39849742, -20.39246027, -20.38638287, -20.38051224,
       -20.37466763, -20.36986365, -20.36419243, -20.35973795,
       -20.35443214, -20.35024406, -20.34610169, -20.34123372,
       -20.33764199, -20.33414959, -20.3308798 , -20.32792954,
       -20.32506967, -20.32258096, -20.32117507,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  